## Imports des bibliothèques nécessaires

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Outils de validation / découpage de scikit-learn
from sklearn.model_selection import (
    train_test_split,     # découpage hold-out train / test
    StratifiedKFold,      # K-fold en respectant les proportions de classes
    cross_val_predict     # prédictions en validation croisée (toutes les obs.)
)

# Prétraitement : standardisation des variables explicatives
from sklearn.preprocessing import StandardScaler

# Pipeline : chaîner prétraitement + modèle dans un seul objet
from sklearn.pipeline import Pipeline

# Classifieurs linéaires vus en cours :
from sklearn.linear_model import Perceptron, LogisticRegression  # perceptron + régression logistique
from sklearn.svm import LinearSVC                               # SVM linéaire (hyperplan à large marge)

# Réseau de neurones (perceptron multicouche / MLP)
from sklearn.neural_network import MLPClassifier

# Métriques d'évaluation
from sklearn.metrics import (
    accuracy_score,             # probabilité de classification correcte (P_test dans le cours)
    classification_report,      # précision / rappel / F1 par classe
    confusion_matrix,           # matrice de confusion numérique
    ConfusionMatrixDisplay      # outil pratique pour tracer la matrice de confusion
)

## Chargement des données et découpage hold-out

### Chargement du fichier de données nettoyées (préparé dans le notebook d'EDA)

In [4]:
df = pd.read_csv("../data/data_clean.csv")

# Suppression éventuelle d'une colonne d'index technique qui ne doit pas être utilisée comme feature
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

# Séparation entre X : matrice des variables explicatives (toutes les colonnes sauf 'diagnosis'), et y : vecteur des étiquettes (diagnostic 0/1)
X = df.drop(columns=["diagnosis"])
y = df["diagnosis"]

# Découpage hold-out : 80 % des données pour l'entraînement, 20 % pour le test
# On stratifie sur y pour conserver la proportion bénin/malin dans les deux sous-échantillons
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,       # environ 20 % pour l'ensemble de test
    random_state=0,      # graine pour rendre le découpage reproductible
    stratify=y           # stratification sur la variable cible
)

print("Nombre total d'exemples :", X.shape[0])
print("Taille ensemble d'entraînement :", X_train.shape[0])
print("Taille ensemble de test        :", X_test.shape[0])

Nombre total d'exemples : 569
Taille ensemble d'entraînement : 455
Taille ensemble de test        : 114
